In [25]:
%matplotlib qt
from pathlib import Path
import mne
import hcp
import hcp.preprocessing as preproc

In [ ]:
## 1. dl the MEG anatomy folder
## 2. dl the structural extended preprocess
## 3. dl the processed MEG 
## 4. dl the unprocessed MEG

## create folder structure
## 1. create trans folder if its not there and make it writable
## 2. check if I have MRI folder
## 3. per subject move T1w folder from struc ext preprocess to MEG/anatomy folder 
## 4. run hcp.make_mne_anatomy to create subjects_dir

subject = '100307'
run_index = 1
data_type = 'rest'
hcp_path = Path('/Volumes/Extreme_SSD/payam_data/hcp')
hcp_path_proc = hcp_path / 'processed'
hcp_path_unproc = hcp_path / 'unprocessed'

kwargs_unproc = {
                "subject": subject,
                "run_index": 1,
                "data_type": "rest",
                "hcp_path": hcp_path / "unprocessed"
                }

kwargs_proc = {
                "subject": subject,
                "run_index": 1,
                "data_type": "rest",
                "hcp_path": hcp_path / "processed"
                }

# create subjects_dir
hcp.make_mne_anatomy(subject=subject,
                    subjects_dir=hcp_path / 'subjects',
                    hcp_path=hcp_path / 'mri',
                    recordings_path=hcp_path / 'trans'
                    )

## read necessary information
info = hcp.read_info(**kwargs_unproc)
annots = hcp.read_annot(**kwargs_proc)
ica = hcp.read_ica(**kwargs_proc)
raw = hcp.read_raw(**kwargs_unproc)
raw.load_data()

## preprocess
preproc.apply_ref_correction(raw)

bad_seg = (annots['segments']['all']) / raw.info['sfreq']
annotations = mne.Annotations(
                                bad_seg[:, 0], (bad_seg[:, 1] - bad_seg[:, 0]),
                                description='bad'
                                )
raw.annotations = annotations
raw.info['bads'].extend(annots['channels']['all'])
raw.pick_types(meg=True, ref_meg=False)

## filter
# l_freq, h_freq = 
raw.filter(l_freq, h_freq, method='iir', iir_params=dict(order=4, ftype='butter'), n_jobs=1, verbose=False)

## ica
exclude = [ii for ii in range(annots['ica']['total_ic_number'][0]) if ii not in annots['ica']['brain_ic_vs']]
preproc.apply_ica_hcp(raw, ica_mat=ica, exclude=exclude)

## setup source space

reading extended structural processing ...
reading RAS freesurfer transform
Combining RAS transform and coregistration
extracting head model
coregistring head model to MNE-HCP coordinates


FileExistsError: Destination file exists. Please use option "overwrite=True" to force overwriting.